In [0]:
!pip install -U keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [0]:
!pip install git+https://github.com/JihongJu/keras-resnet3d.git

  Cloning https://github.com/JihongJu/keras-resnet3d.git to /tmp/pip-req-build-nb9xuk46
  Running command git clone -q https://github.com/JihongJu/keras-resnet3d.git /tmp/pip-req-build-nb9xuk46
  Running command git submodule update --init --recursive -q
  Created wheel for keras-resnet3d: filename=keras_resnet3d-0.0.1-cp36-none-any.whl size=4773 sha256=0f727f975a6a17885dc488b03949afd382eed1617e4dba19e14839831f18984f
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ddt6qmy/wheels/98/fa/00/0dfe23f54c8f40a14c53f355effb4b198f7727d6d7f05be8b1
Successfully built keras-resnet3d


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import keras
from keras.layers import Dense,Conv2D,LSTM,Bidirectional,TimeDistributed,Input,RepeatVector
from keras.models import Model
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [0]:
#Importing the Data 
X_train = np.load('/content/drive/My Drive/Sign Language Recognition/X_train.npy')
y_train = np.load('/content/drive/My Drive/Sign Language Recognition/y_train.npy')
print(X_train.shape)
print(y_train.shape)

(38, 24, 8, 224, 224, 3)
(38, 28)


In [0]:
input_shape = (38,24,8,224,224,3)
out_timesteps = 28
out_vocab = 1232

In [0]:
# Defining the resnet3d model
from resnet3d import Resnet3DBuilder
resnet3dmodel = Resnet3DBuilder.build_resnet_18((input_shape[2], input_shape[3], input_shape[4], input_shape[5]), 0)
# layer_names = [layer.name for layer in model.layers]
last_layer = resnet3dmodel.layers[-2]
base_model = Model(inputs = resnet3dmodel.input, outputs=last_layer.output)

In [0]:
# Defining the main model

#Input layer
input_tensor = Input(shape=(input_shape[1],input_shape[2],input_shape[3],input_shape[4],input_shape[5]))

#Feature Extractor
resnet = TimeDistributed(base_model)(input_tensor)

#Encoder
bidirectional_lstm_1 = Bidirectional(LSTM(1024))(resnet)

#Decoder
repeatVector = RepeatVector(out_timesteps)(bidirectional_lstm_1)
lstm_1 = LSTM(1024, return_sequences=True)(repeatVector)

#Output Dense layer
output_tensor = Dense(out_vocab, activation='softmax')(lstm_1)
print(output_tensor)

model = Model(inputs=input_tensor,outputs=output_tensor)

Tensor("dense_2/truediv:0", shape=(None, 28, 1232), dtype=float32)


In [0]:
#Model compilation
rms = RMSprop(lr=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [0]:
# filename = 'model.h1.6_april_20'
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# train model
history = model.fit(X_train, y_train.reshape(y_train.shape[0], y_train.shape[1], 1),
                    epochs=30, batch_size=2, validation_split = 0.2, 
                    verbose=1,shuffle=True)

Train on 30 samples, validate on 8 samples
Epoch 1/30
30/30 [==============================] - 69s 2s/step - loss: 4.7459 - accuracy: 0.5321 - val_loss: 4.0064 - val_accuracy: 0.5223
Epoch 2/30
30/30 [==============================] - 55s 2s/step - loss: 3.1683 - accuracy: 0.5750 - val_loss: 3.5090 - val_accuracy: 0.5223
Epoch 3/30
30/30 [==============================] - 55s 2s/step - loss: 2.9547 - accuracy: 0.5798 - val_loss: 3.5350 - val_accuracy: 0.5312
Epoch 4/30
30/30 [==============================] - 55s 2s/step - loss: 2.7319 - accuracy: 0.5821 - val_loss: 3.5303 - val_accuracy: 0.5312
Epoch 5/30
30/30 [==============================] - 55s 2s/step - loss: 2.6044 - accuracy: 0.5821 - val_loss: 3.4279 - val_accuracy: 0.5312
Epoch 6/30
30/30 [==============================] - 55s 2s/step - loss: 2.5729 - accuracy: 0.5821 - val_loss: 3.4916 - val_accuracy: 0.5312
Epoch 7/30
30/30 [==============================] - 55s 2s/step - loss: 2.4283 - accuracy: 0.5833 - val_loss: 3.7922 